In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams["font.family"] = "Times New Roman"

from utils import get_hdf_keys
from train import training
from evaluation import evaluate, get_run_results
import warnings
import pickle
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

/Users/nikolaushouben/opt/anaconda3/envs/Wattcast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set working directory
import wandb
wandb.login()

os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [3]:
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

In [4]:
locations_dict

{'GLENDOVEER.h5': ['13596.MWh',
  '13597.MWh',
  '13598.MWh',
  '13599.MWh',
  'CLIFFGATE.MWh',
  'NORTHEAST.MWh'],
 'LENTS.h5': ['13101.MWh', 'HAPPY.VALLEY.MWh', 'MT.SCOTT.MWh', 'NORTH.MWh'],
 'MIDWAY.h5': ['DIVISION.MWh', 'DOUGLAS.MWh', 'LYNCH.MWh', 'POWELLHURST.MWh'],
 'RAMAPO.h5': ['EMERALD.MWh', 'GILBERT.MWh', 'RAMAPO.13.MWh'],
 'KELLY.BUTTE.h5': ['BINNSMEAD.MWh',
  'FAIRLAWN.MWh',
  'MALL.205.MWh',
  'MCGREW.MWh']}

### Loop through all locations and resolutions

### Run for a single location

In [ ]:
init_config = {
    'spatial_scale': "GLENDOVEER",
    'temp_resolution': 60,
    'location': '13596.MWh',
    'models_to_train': ["xgb"],
    'horizon_in_hours': 48,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather_available': True,
    'datetime_encodings': True,
    'heat_wave_binary': True,
    'datetime_attributes': ["dayofweek", "week"],
    'use_auxilary_data': True # to use data from other locations on the same scale (in case there is not enough training data)
}

name_id = (
    init_config["spatial_scale"]
    + "_"
    + init_config["location"]
    + "_"
    + str(init_config["temp_resolution"])
    + "min" 
    + "_"
    + "aux_data-"
    +str(init_config['use_auxilary_data'])
)
wandb.init(
    project="Portland_AMI", name=name_id, id=name_id
) 

config, models = training(init_config)

eval_dict = evaluate(config, models)

df_metrics = get_run_results(eval_dict, config)

wandb.finish()

## Debugging